# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
vaca_data = pd.read_csv("../weather_data.csv", dtype = "object", encoding="utf-8")

vaca_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mehrān,0,IR,1600977165,28,33.12,46.16,68.0,2.24
1,Busselton,91,AU,1600977168,54,-33.65,115.33,55.0,18.05
2,Bulgan,100,MN,1600977172,77,48.81,103.53,48.22,2.73
3,Bethel,1,US,1600977175,47,41.37,-73.41,75.99,8.05
4,Mar del Plata,90,AR,1600977178,93,-38.0,-57.56,48.2,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [8]:
locations = vaca_data[["Lat", "Lng"]].astype(float)

humidty = vaca_data["Humidity"].astype(float)

vaca_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mehrān,0,IR,1600977165,28,33.12,46.16,68.0,2.24
1,Busselton,91,AU,1600977168,54,-33.65,115.33,55.0,18.05
2,Bulgan,100,MN,1600977172,77,48.81,103.53,48.22,2.73
3,Bethel,1,US,1600977175,47,41.37,-73.41,75.99,8.05
4,Mar del Plata,90,AR,1600977178,93,-38.0,-57.56,48.2,10.29


In [9]:
fig = gmaps.figure( zoom_level = 2, center = (-33.93,18.42))

heat_layer = gmaps.heatmap_layer(locations, weights=humidty * 50, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
vaca_data.dtypes

vaca_max_temp = vaca_data.loc[(vaca_data['Max Temp'].astype(float) <= 80) & (vaca_data['Max Temp'].astype(float) > 70)]
vaca_wind = vaca_max_temp.loc[(vaca_max_temp['Wind Speed'].astype(float) <= 10)]
vaca_cloudiness = vaca_wind.loc[(vaca_wind['Cloudiness'].astype(float) <= 4)]

vaca_ideal_df = vaca_cloudiness.dropna()
vaca_ideal_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,Bethel,1,US,1600977175,47,41.37,-73.41,75.99,8.05
12,Pacific Grove,1,US,1600976977,77,36.62,-121.92,75.99,5.82
26,Castro,0,BR,1600977228,41,-24.79,-50.01,74.23,5.57
27,Al Bardīyah,0,LY,1600977254,65,31.76,25.09,76.96,4.05
60,Morondava,0,MG,1600977363,80,-20.28,44.28,72.63,5.17
86,Noumea,0,NC,1600977149,77,-22.28,166.46,72.0,1.12
129,Safi,4,MA,1600977593,78,32.3,-9.24,71.69,8.59
180,Harnai,0,PK,1600977764,17,30.1,67.94,77.83,4.21
206,Anzio,0,IT,1600977851,89,41.49,12.62,73.0,1.99
211,Rawson,0,AR,1600977867,19,-43.3,-65.1,74.82,8.63


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = vaca_ideal_df
hotel_df["Hotel Name"] = ""
hotel_df 

/Users/davidruppel/opt/anaconda3/envs/bcs/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
3,Bethel,1,US,1600977175,47,41.37,-73.41,75.99,8.05,
12,Pacific Grove,1,US,1600976977,77,36.62,-121.92,75.99,5.82,
26,Castro,0,BR,1600977228,41,-24.79,-50.01,74.23,5.57,
27,Al Bardīyah,0,LY,1600977254,65,31.76,25.09,76.96,4.05,
60,Morondava,0,MG,1600977363,80,-20.28,44.28,72.63,5.17,
86,Noumea,0,NC,1600977149,77,-22.28,166.46,72.0,1.12,
129,Safi,4,MA,1600977593,78,32.3,-9.24,71.69,8.59,
180,Harnai,0,PK,1600977764,17,30.1,67.94,77.83,4.21,
206,Anzio,0,IT,1600977851,89,41.49,12.62,73.0,1.99,
211,Rawson,0,AR,1600977867,19,-43.3,-65.1,74.82,8.63,


In [12]:
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"


# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i,r in hotel_df.iterrows():
    target_lat = r['Lat']
    target_lon = r['Lng']
    target_coordinates = f'{target_lat}, {target_lon}'
    
    # set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    places_data = response.json()

    # Print the json (pretty printed)
    try:
        print(places_data['results'][0]['name'])
        hotel_df.loc[i,'Hotel Name'] = places_data['results'][0]['name']
    except IndexError:
        print("Hotel not found")
        


Courtyard by Marriott Danbury


/Users/davidruppel/opt/anaconda3/envs/bcs/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/davidruppel/opt/anaconda3/envs/bcs/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Hyatt Regency Monterey Hotel And Spa On Del Monte Golf Course
Urano Business Hotel
Hotel Burdy
Palissandre Cote Ouest resort & SPA
Hilton Noumea La Promenade Residences
RIAD LE CHEVAL BLANC - HÔTEL
Hotel not found
Casa Isa
Hotel Punta León
Hotel not found
Holiday Inn Express & Suites Price
Constance Resort Garden - Blu Hotels
Hotel not found
Hampton Inn Seymour
Walkabout Lodge & Tavern
Tuyuni Guest House
Padadita Beach Hotel
Salinas do Maragogi All Inclusive Resort
Latitude20 Roebourne Village


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [14]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
hotel_df.dtypes

City          object
Cloudiness    object
Country       object
Date          object
Humidity      object
Lat           object
Lng           object
Max Temp      object
Wind Speed    object
Hotel Name    object
dtype: object